# Performing Model Selection Using Topic Coherence

This notebook will perform topic modeling on the 20 Newsgroups corpus using LDA. We will perform model selection (over the number of topics) using topic coherence as our evaluation metric. This will showcase some of the features of the topic coherence pipeline implemented in `gensim`. In particular, we will see several features of the `CoherenceModel`.

In [1]:
from __future__ import print_function

import os
import re
from collections import OrderedDict

from gensim.corpora import TextDirectoryCorpus, MmCorpus
from gensim import utils, models

Using TensorFlow backend.


## Parsing the Dataset

The 20 Newsgroups dataset uses a hierarchical directory structure to store the articles. The structure looks something like this:
```
20news-18828/
|-- alt.atheism
|   |-- 49960
|   |-- 51060
|   |-- 51119
|-- comp.graphics
|   |-- 37261
|   |-- 37913
|   |-- 37914
|-- comp.os.ms-windows.misc
|   |-- 10000
|   |-- 10001
|   |-- 10002
```

The files are in the newsgroup markup format, which includes some headers, quoting of previous messages in the thread, and possibly PGP signature blocks. The message body itself is raw text, which requires preprocessing. The code immediately below is an adaptation of [an active PR](https://github.com/RaRe-Technologies/gensim/pull/1388) for parsing hierarchical directory structures into corpora. The code just below that builds on this basic corpus parser to handle the newsgroup-specific text parsing.

In [2]:
class NewsgroupCorpus(TextDirectoryCorpus):
    """Parse 20 Newsgroups dataset."""

    def getstream(self):
        for text in super(NewsgroupCorpus, self).getstream():
            yield utils.to_unicode(text, encoding='latin')

    def preprocess_text(self, text):
        body = extract_body(text)
        return super(NewsgroupCorpus, self).preprocess_text(body)

    
def extract_body(text):
    return strip_newsgroup_header(
        strip_newsgroup_footer(
            strip_newsgroup_quoting(text)))


def strip_newsgroup_header(text):
    """Given text in "news" format, strip the headers, by removing everything
    before the first blank line.
    """
    _before, _blankline, after = text.partition('\n\n')
    return after


_QUOTE_RE = re.compile(r'(writes in|writes:|wrote:|says:|said:'
                       r'|^In article|^Quoted from|^\||^>)')
def strip_newsgroup_quoting(text):
    """Given text in "news" format, strip lines beginning with the quote
    characters > or |, plus lines that often introduce a quoted section
    (for example, because they contain the string 'writes:'.)
    """
    good_lines = [line for line in text.split('\n')
                  if not _QUOTE_RE.search(line)]
    return '\n'.join(good_lines)


_PGP_SIG_BEGIN = "-----BEGIN PGP SIGNATURE-----"
def strip_newsgroup_footer(text):
    """Given text in "news" format, attempt to remove a signature block."""
    try:
        return text[:text.index(_PGP_SIG_BEGIN)]
    except ValueError:
        return text

### Loading the Dataset

Now that we have defined the necessary code for parsing the dataset, let's load it up and serialize it into Matrix Market format. We'll do this because we want to train LDA on it with several different parameter settings, and this will allow us to avoid repeating the preprocessing.

In [3]:
# Replace data_path with path to your own copy of the corpus.
# You can download it from here: http://qwone.com/~jason/20Newsgroups/
# I'm using the original, called: 20news-19997.tar.gz

home = os.path.expanduser('~')
data_dir = os.path.join(home, 'workshop', 'nlp', 'data')
data_path = os.path.join(data_dir, '20_newsgroups')

In [4]:
%%time

corpus = NewsgroupCorpus(data_path)
corpus.dictionary.filter_extremes(no_below=5, no_above=0.8)
dictionary = corpus.dictionary
print(len(corpus))
print(dictionary)

19998
Dictionary(24593 unique tokens: [u'woods', u'hanging', u'woody', u'localized', u'gaa']...)
CPU times: user 20 s, sys: 797 ms, total: 20.8 s
Wall time: 21.1 s


In [5]:
%%time

mm_path = os.path.join(data_dir, '20_newsgroups.mm')
MmCorpus.serialize(mm_path, corpus, id2word=dictionary)
mm_corpus = MmCorpus(mm_path)  # load back in to use for LDA training

CPU times: user 25.1 s, sys: 1.09 s, total: 26.1 s
Wall time: 26.6 s


## Training the Models

Our goal is to determine which number of topics produces the most coherent topics for the 20 Newsgroups corpus. The corpus is roughly 20,000 documents. If we used 100 topics and the documents were evenly distributed among topics, we'd have clusters of 200 documents. This seems like a reasonable upper bound. In this case, the corpus actually has categories, defined by the first-level directory structure. This can be seen in the directory structure shown above, and three examples are: `alt.atheism`, `comp.graphics`, and `comp.os.ms-windows.misc`. There are 20 of these (hence the name of the dataset), so we'll use 20 as our lower bound for the number of topics.

One could argue that we already know the model should have 20 topics. I'll argue there may be additional categorizations within each newsgroup and we might hope to capture those by using more topics. We'll step by increments of 10 from 20 to 100.

In [13]:
%%time

trained_models = OrderedDict()
for num_topics in range(20, 101, 10):
    print("Training LDA(k=%d)" % num_topics)
    lda = models.LdaMulticore(
        mm_corpus, id2word=dictionary, num_topics=num_topics, workers=4,
        passes=10, iterations=100, random_state=42, eval_every=None,
        alpha='asymmetric',  # shown to be better than symmetric in most cases
        decay=0.5, offset=64  # best params from Hoffman paper
    )
    trained_models[num_topics] = lda

Training LDA(k=20)
Training LDA(k=30)
Training LDA(k=40)
Training LDA(k=50)
Training LDA(k=60)
Training LDA(k=70)
Training LDA(k=80)
Training LDA(k=90)
Training LDA(k=100)
CPU times: user 20min 42s, sys: 3min 13s, total: 23min 56s
Wall time: 23min 54s


In [6]:
# Some useful utility functions in case you want to save your models.

models_dir = os.path.join(home, 'workshop', 'nlp', 'models')

def save_models(named_models):
    for num_topics, model in named_models.items():
        model_path = os.path.join(models_dir, 'lda-newsgroups-k%d.lda' % num_topics)
        model.save(model_path, separately=False)

        
def load_models():
    trained_models = OrderedDict()
    for num_topics in range(20, 101, 10):
        model_path = os.path.join(models_dir, 'lda-newsgroups-k%d.lda' % num_topics)
        print("Loading LDA(k=%d) from %s" % (num_topics, model_path))
        trained_models[num_topics] = models.LdaMulticore.load(model_path)

    return trained_models

In [ ]:
# save_models(trained_models)
# trained_models = load_models()

## Evaluation Using Coherence

Now we get to the heart of this notebook. In this section, we'll evaluate each of our LDA models using topic coherence. Coherence is a measure of how interpretable the topics are to humans. It is based on the representation of topics as the top-N most probable words for a particular topic. More specifically, given the topic-term matrix for LDA, we sort each topic from highest to lowest term weights and then select the first N terms.

Coherence essentially measures how similar these words are to each other. There are various methods for doing this, most of which have been explored in the paper ["Exploring the Space of Topic Coherence Measures"](https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf). The authors performed a comparative analysis of various methods, correlating them to human judgements. The method named "c_v" coherence was found to be the most highly correlated. This and several of the other methods have been implemented in `gensim.models.CoherenceModel`. We will use this to perform our evaluations.

The "c_v" coherence method makes an expensive pass over the corpus, accumulating term occurrence and co-occurrence counts. It only accumulates counts for the terms in the lists of top-N terms for each topic. In order to ensure we only need to make one pass, we'll construct a "super topic" from the top-N lists of each of the models. This will consist of a single topic with all the relevant terms from all the models. We choose 20 as N.

In [8]:
%%time

# Now estimate the probabilities for the CoherenceModel.
# This performs a single pass over the reference corpus, accumulating
# the necessary statistics for all of the models at once.
cm = models.CoherenceModel.for_models(
    trained_models.values(), dictionary, texts=corpus.get_texts(), coherence='c_v')

CPU times: user 41.3 s, sys: 2.42 s, total: 43.7 s
Wall time: 1min 26s


In [10]:
%%time

coherence_estimates = cm.compare_models(trained_models.values())
coherences = dict(zip(trained_models.keys(), coherence_estimates))

CPU times: user 1min 9s, sys: 441 ms, total: 1min 9s
Wall time: 1min 10s


In [11]:
def print_coherence_rankings(coherences):
    avg_coherence = \
        [(num_topics, avg_coherence)
         for num_topics, (_, avg_coherence) in coherences.items()]
    ranked = sorted(avg_coherence, key=lambda tup: tup[1], reverse=True)
    print("Ranked by average '%s' coherence:\n" % cm.coherence)
    for item in ranked:
        print("num_topics=%d:\t%.4f" % item)
    print("\nBest: %d" % ranked[0][0])

In [12]:
print_coherence_rankings(coherences)

Ranked by average 'c_v' coherence:

num_topics=20:	0.5614
num_topics=50:	0.5522
num_topics=30:	0.5385
num_topics=80:	0.5286
num_topics=40:	0.5239
num_topics=70:	0.5189
num_topics=100:	0.5140
num_topics=60:	0.5090
num_topics=90:	0.4824

Best: 20


### Results so Far

So far in this notebook, we have used `gensim`'s `CoherenceModel` to perform model selection over the number of topics for LDA. We found that for the 20 Newsgroups corpus, 20 topics is indeed best. We showcased the ability of the coherence pipeline to evaluate individual aggregated model coherence (Note that the individual topic coherence is also computed by `cm.compare_models`). We also demonstrated how to avoid repeated passes over the corpus, estimating the term similarity probabilities for all relevant terms just once. Topic coherence is a powerful alternative to evaluation using perplexity on a held-out document set. It is appropriate to use whenever the objective of the topic modeling is to present the topics as top-N lists for human consumption.

Note that coherence calculations are generally much more accurate when a larger reference corpus is used to estimate the probabilities. In this case, we used the same corpus as for our modeling, which is relatively small at only 20,000 documents. A better reference corpus is the full Wikipedia corpus. The motivated explorer of this notebook is encouraged to download that corpus (see [Experiments on the English Wikipedia](https://radimrehurek.com/gensim/wiki.html)) and use it for probability estimation.

Next we'll look at another method of coherence evaluation using distributional word embeddings.

### Evaluating Coherence with Word2Vec

The fact that "c_v" coherence uses distributional semantics to evalaute word similarity motivates the use of Word2Vec for coherence evaluation. This idea is explored further in an appendix at the end of the notebook. The `CoherenceModel` implemented in `gensim` also supports this, so let's look at a few examples.

In [16]:
%%time

class TextsIterable(object):
    """Wrap a TextCorpus in something that yields texts from its __iter__.
    
    It's necessary to use this because the Word2Vec model is built by scanning
    over the texts several times. Passing in corpus.get_texts() would result in
    an empty iterable on passes after the first.
    """
    
    def __init__(self, corpus):
        self.corpus = corpus
    
    def __iter__(self):
        return self.corpus.get_texts()


cm = models.CoherenceModel.for_models(
    trained_models.values(), dictionary, texts=TextsIterable(corpus), coherence='c_w2v')

CPU times: user 3min 25s, sys: 5.71 s, total: 3min 31s
Wall time: 2min 11s


In [17]:
%%time

coherence_estimates = cm.compare_models(trained_models.values())
coherences = dict(zip(trained_models.keys(), coherence_estimates))
print_coherence_rankings(coherences)

Ranked by average 'c_w2v' coherence:

num_topics=80:	0.7491
num_topics=100:	0.7455
num_topics=90:	0.7388
num_topics=70:	0.7327
num_topics=50:	0.7301
num_topics=60:	0.7285
num_topics=40:	0.7136
num_topics=30:	0.7061
num_topics=20:	0.6865

Best: 80
CPU times: user 2.55 s, sys: 98.3 ms, total: 2.65 s
Wall time: 2.65 s


#### Using pre-trained word vectors for coherence evaluation.

Whoa! These results are completely different from those of the "c_v" method, and "c_w2v" is saying the model we thought was best is actually the worst! So what happened here?

The same note must be made for Word2Vec ("c_w2v") that we made for "c_v": results are more accurate when a larger reference corpus is used. Except for "c_w2v", this is actually _way, way_ more important. Distributional word embedding techniques such as Word2Vec are fitting a probability distribution with a large number of parameters, and doing that takes a lot of data.

Luckily, there are a variety of pre-trained word vectors [freely available for download](http://ahogrammer.com/2017/01/20/the-list-of-pretrained-word-embeddings/). Below we demonstrate using word vectors trained on ~100 billion words from Google News, [available at this link](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing). Note that this file is 1.5G, so downloading it can take quite some time. It is also quite slow to load and ends up occupying about 3.35G in memory (this load time is included in the timing below). There is no need to use such a large set of word vectors for this evaluation; this one is just readily available.

In [32]:
%%time

models_dir = os.path.join(home, 'workshop', 'nlp', 'models')
vectors_path = os.path.join(models_dir, 'GoogleNews-vectors-negative300.bin.gz')
keyed_vectors = models.KeyedVectors.load_word2vec_format(vectors_path, binary=True)

# still need to estimate_probabilities, but corpus is not scanned
cm = models.CoherenceModel.for_models(
    trained_models.values(), dictionary, texts=corpus.get_texts(),
    coherence='c_w2v', keyed_vectors=keyed_vectors)

CPU times: user 3min 5s, sys: 8.26 s, total: 3min 13s
Wall time: 3min 15s


In [33]:
%%time

coherence_estimates = cm.compare_models(trained_models.values())
coherences = dict(zip(trained_models.keys(), coherence_estimates))
print_coherence_rankings(coherences)

Ranked by average 'c_w2v' coherence:

num_topics=50:	0.5004
num_topics=40:	0.4988
num_topics=30:	0.4966
num_topics=60:	0.4959
num_topics=20:	0.4943
num_topics=70:	0.4920
num_topics=80:	0.4882
num_topics=90:	0.4821
num_topics=100:	0.4803

Best: 50
CPU times: user 5.21 s, sys: 1.22 s, total: 6.43 s
Wall time: 5.78 s


#### Watch out for Out-of-Vocabulary (OOV) terms!

At first glance, it might seem like the "c_w2v" coherence with the GoogleNews vectors disagrees quite a bit with the "c_v" method and doesn't seem to improve on training on the newsgroups corpus. If you run the above code with debug logging enabled, you'll notice a TON of logging messages stating something like "at least one topic word not in word2vec model." This is a real gotcha to watch out for! Pre-trained word vectors means there is likely to be some vocabulary mismatch. So unless the corpus you're modeling on was included in the training data for the vectors, you need to watch out for this. In the results above, it is easy to diagnose because all of the models have very similar coherence rankings. You can use the function below to dig in and see exactly how bad the issue is.

In [34]:
import itertools


def report_on_oov_terms(cm, topic_models):
    """OOV = out-of-vocabulary"""
    vectors = cm._accumulator.model
    topics_as_topn_terms = [
        models.CoherenceModel.top_topics_as_word_lists(model, dictionary)
        for model in topic_models
    ]

    super_topic = set()
    for topic in topics_as_topn_terms:
        super_topic.update(itertools.chain.from_iterable(topic))

    oov_words = [word for word in super_topic if word not in vectors.vocab]
    print('number of oov words: %d' % len(oov_words))
    for num_topics, words in zip(trained_models.keys(), topics_as_topn_terms):
        flat_words = itertools.chain.from_iterable(words)
        oov_words = [word for word in flat_words if word not in vectors.vocab]
        print('number of oov words for num_topics=%d: %d' % (num_topics, len(oov_words)))

report_on_oov_terms(cm, trained_models.values())

number of oov words: 407
number of oov words for num_topics=20: 19
number of oov words for num_topics=30: 54
number of oov words for num_topics=40: 81
number of oov words for num_topics=50: 99
number of oov words for num_topics=60: 121
number of oov words for num_topics=70: 136
number of oov words for num_topics=80: 201
number of oov words for num_topics=90: 181
number of oov words for num_topics=100: 198


*Yikes!* That's a lot of terms that are being ignored when calculating the coherence metrics. So these results are not really reliable. Let's use a different set of pre-trained word vectors. I trained these on a recent Wikipedia dump using skip-gram negative sampling (SGNS) with a context window of 5 and 300 dimensions.

In [26]:
vectors_path = os.path.join(models_dir, 'wiki-en_sgns5-w5-s300.bin.gz')
keyed_vectors = models.KeyedVectors.load_word2vec_format(vectors_path, binary=True)

# still need to estimate_probabilities, but corpus is not scanned
cm = models.CoherenceModel.for_models(
    trained_models.values(), dictionary, texts=corpus.get_texts(),
    coherence='c_w2v', keyed_vectors=keyed_vectors)

In [27]:
%%time

coherence_estimates = cm.compare_models(trained_models.values())
coherences = dict(zip(trained_models.keys(), coherence_estimates))
print_coherence_rankings(coherences)

Ranked by average 'c_w2v' coherence:

num_topics=20:	0.5812
num_topics=30:	0.5804
num_topics=50:	0.5761
num_topics=40:	0.5740
num_topics=60:	0.5681
num_topics=70:	0.5634
num_topics=80:	0.5596
num_topics=100:	0.5567
num_topics=90:	0.5559

Best: 20
CPU times: user 3.2 s, sys: 244 ms, total: 3.44 s
Wall time: 3.36 s


In [31]:
report_on_oov_terms(cm, trained_models.values())

number of oov words: 80
number of oov words for num_topics=20: 1
number of oov words for num_topics=30: 9
number of oov words for num_topics=40: 17
number of oov words for num_topics=50: 9
number of oov words for num_topics=60: 19
number of oov words for num_topics=70: 24
number of oov words for num_topics=80: 30
number of oov words for num_topics=90: 33
number of oov words for num_topics=100: 26


#### Looks like we've now restored order

The results with the Wikipedia-trained word vectors are much better because we have less terms OOV. The "c_w2v" evalution is now agreeing with "c_v" on the best model, and the rest of the ordering is generally similar. Note that the "c_w2v" values should not be compared directly to those produced by the "c_v" method. Only the ranking of models is comparable.

## Appendix: Why Word2Vec for Coherence?

The "c_v" coherence method drags a sliding window across all documents in the corpus to accumulate co-occurrence statistics. Similarity is calculated using normalized pointwise mutual information (PMI) values estimated from these statistics. More specifically, each word is represented by a vector of its NPMI with every other word in its top-N topic list. These vectors are then used to compute (cosine) similarity between words. The restriction to the other words in the top-N list was found to produce better results than using the entire vocabulary and other methods of reducing the vocabulary (see section 3.2.2 of http://www.aclweb.org/anthology/W13-0102).

The fact that a reduced space is superior for these metrics indicates there is noise getting in the way. The "c_v" method can be seen as constructing an NPMI matrix between words. The vector of NPMI values for a particular word can then be looked up by indexing the row or column corresponding to that word's `Dictionary` ID. The reduction to the "topic word space" can then be achieved by using a mask to select out the top-N topic words. If we are constructing an NPMI matrix between words, then discarding some elements to reduce noise, why not factorize the matrix instead? Dimensionality reduction techniques such as SVD do a great job of reducing noise along with dimensionality, while also providing a compressed representation to work with.

[Recent work](https://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization) has shown that Word2Vec (trained with Skip-Gram Negative Sampling (SGNS)) is actually implicitly factorizing a PMI matrix shifted by a positive constant. [A subsequent paper](http://dl.acm.org/citation.cfm?id=2914720) compared Word2Vec to a few different PMI-based metrics and showed that it found coherence values that correlated more strongly with human judgements.